# 13 - Editing Existing Molecules

### In this tutorial we will learn how to edit existing molecules with architector functions.

**(A)** How load exisiting molecules in and prepare to edit.

**(B)** How to remove atoms from a molecule.

**(C)** How add functional groups to existing molecules.

### For (A) convert_io_molecules is a singularly useful utility!

In [ ]:
from architector import (convert_io_molecule,
                         build_complex,
                         view_structures)

In [ ]:
# First, let's build an example molecule to evaluate
inp = {'core':{'metal':'Fe','coreType':'octahedral'},
               'ligands':['bipy'], # One bipyridine molecule with waters.
                'parameters':{'assemble_method':'UFF', # Use UFF for SPEED.
                              # Will still relax with GFn2-xTB!
                             }}
out = build_complex(inp)
# Takes ~10 seconds on my laptop

In [ ]:
# Now, we can see this molecule
view_structures(out)
out

In [ ]:
# And pull out the structure back to an architector molecule using convert_io_molecule
arch_mol = convert_io_molecule(out[list(out.keys())[0]]['mol2string'])

In [ ]:
# Since this was generated with architector, it already has spin/charge assigned, and will use these for calculations.
print('Num Unpaired Eelectrons: ',arch_mol.uhf, 
      'Molecule Charge: ', arch_mol.charge)

In [ ]:
# But, a more realistic case for this functionality would be loaded something relaxed with an external DFT code and interpreting with Architector.
# Here, we can output just an xyz file:
arch_mol.write_xyz('example.xyz')

In [ ]:
# And we can look at the xyz file to show it is normal:
!head example.xyz

In [ ]:
# We can load the molecule back into Architector with a single line.
newmol = convert_io_molecule('example.xyz')
# And detect the molecular graph, charge, and spin with a single line!
newmol.create_mol_graph(skin=0.35) # Skin tells the the size beyond the sum of covalent radii to expand to look for neighbors in Angstroms)
# One can increase or decrease if bonds are missed/added. Default is 0.2 Angstroms, which was too small for this case!
newmol.detect_charge_spin() # Detect the charge a spin of the molecule -> Currently works best for mononuclear metal complexes.

In [ ]:
# Note that the exact charge and spin was recovered without the graph/values needed.
print('New Num Unpaired Eelectrons: ',newmol.uhf, 
      'New Molecule Charge: ', newmol.charge)

In [ ]:
# Note, however, that bond orders are slightly different:
view_structures(newmol)

### Now, For (B), let's remove some atoms!


A useful function that can be performed on a molecule is deleting existing atoms.


But what atoms should we delete?

Let's try deleting the bipy ligand!

In [ ]:
# To start, we have a utility function to divide the molecule into ligands
lig_info = newmol.split_ligs()
print(lig_info)

In [ ]:
# To Delete the bipy, we want to remove all the indices under the "original_lig_inds" key in the first array.
newmol.remove_atoms(lig_info['original_lig_inds'][0])

In [ ]:
# Let's see! 
view_structures(newmol)
# Should look good! 
# Note that this will not track the charge/spin of the molecule with atoms deleted. So if a ligand/functional group deleted is charged/radical, 
# Re-running detect_charge_spin() is suggested!

### Now, for (C) - one of the more important architector structure editing functionalities!

Functionalization! 

Architector now contains powerful 3D functionalization routines that should be easy to use.

Let's start again from the example.xyz!

In [ ]:
# Loading into a new molecule format.
newmol2 = convert_io_molecule('example.xyz')
newmol2.create_mol_graph(skin=0.35)
newmol2.detect_charge_spin()

In [ ]:
# View, this time with indices
view_structures(newmol2, labelinds=True, w=400, h=400)

### Let's add a C8 chain to carbon with index 4, and a C6 chain to oxygen 27.

Here, we use the functionalize_3D function, passing:

1) functional_groups : list of SMILES strings of functional groups or labels of functional groups.
   Note that for functional groups, architector assumes the first atom in the SMILES string is where the 
   functional group will attach to the molecule.
2) functionalization_inds : list of the indices in the molecule where the functional groups will attach!
3) bond_orders : list of the bond orders of the functional group to the corresponding index.
4) xtb_opt : bool : whether to perform a constrained xtb optimization.

Note that this function will always assume the existing molecule atoms will be fixed in space!

They WILL NOT move during functionalization.

In [ ]:
newmol2.functionalize_3D(
    functional_groups=['C'*8,'C'*6], # C8 and C6 smiles strings
    functionalization_inds=[4,27], # Atom 4 (carbon) and atom 27 (oxygen)
    bond_orders=[1,1], # Single bonds
    xtb_opt=True
    )

In [ ]:
# Should look good!
view_structures(newmol2,w=400,h=400,labelinds=True)

In [ ]:
# Now, let's do a secondary functionalization with a named functional group!
from architector.io_ptable import functional_groups_dict
# Print out all the "known" functional group names.
functional_groups_dict.keys()

In [ ]:
# Now, we can do a secondary functionalization with a carboxylate at the other remaining para-carbon position (should be index 9!)
newmol2.functionalize_3D(
    functional_groups=['carboxylate'],
    functionalization_inds=[9],
    # Note - no XTB optimization! - Can result in stranger interatomic angles!
)

In [ ]:
# Should look good! Maybe a bad carbon-carboxylate angle since only UFF used for optimization
view_structures(newmol2,w=400,h=400,labelinds=True)

In [ ]:
# We can also perform ring-closures using multiple coordination indices! :
# Here, we can add ethylene to the briding bipy atoms to form a hex-carbon ring!
newmol2.functionalize_3D(functional_groups=['C=C'], # Ethylene smiles.
                     functionalization_inds=[[5,8]], # Indices in molecule to bind molecule to.
                     functional_group_mol_inds=[[0,1]], # Indices in functional group to add
                     bond_orders=[[1,1]], # Bond orders of ring closure 
                     # No XTB optimization again.
                     )

In [ ]:
view_structures(newmol2,w=400,h=400)

In [ ]:
# Note that the charge of the molecule has shifted with the addition of the charged 
# Carboxylate functional group!
print('New3 Num Unpaired Electrons: ',newmol2.uhf, 
      'New2 Molecule Charge: ', newmol2.charge)

In [ ]:
# Putting together all the components for a full end-to-end example.
newmol2 = convert_io_molecule('example.xyz') # Load molecule from file
newmol2.create_mol_graph(skin=0.35)
newmol2.detect_charge_spin()
view_structures(newmol2,labelinds=True)
newmol2.remove_atoms([22,23]) # Remove two hydrogens from same water
view_structures(newmol2,labelinds=True)
funct_group_mol = convert_io_molecule('CCCCCC')
view_structures(funct_group_mol,labelinds=True) # Create 6-long hydrocarbon chain 
funct_group_mol.remove_atoms([6,18]) # Remove hydrogens from first and last indices.
newmol2.functionalize_3D(functional_groups=[funct_group_mol], 
                         functionalization_inds=[[21,21]], # Bond the hydrocarbons to oxygen we removed indices from.
                         functional_group_mol_inds=[[0,5]],
                         bond_orders=[[1,1]], # Add as single bonds
                         remove_hydrogens_when_adding=[[False,False]], # Since we already removed the hydrogens we don't have 
                         # Have the program do it again!
                         xtb_opt=True # Do XTB optimization.
                         )
print('Final Functionalized Structure:')
view_structures(newmol2)

In [ ]:
# Technically, this can be only a couple-line code as well:
newmol2 = convert_io_molecule('example.xyz') # Load molecule from file
newmol2.create_mol_graph(skin=0.35)
newmol2.detect_charge_spin()
# Now, do the functionalization in one-shot.
newmol2.functionalize_3D(functional_groups=['CCCCCC'], 
                         functionalization_inds=[[21,21]], # Bond the hydrocarbons to oxygen we removed indices from.
                         functional_group_mol_inds=[[0,5]],
                         bond_orders=[[1,1]], # Add as single bonds
                         xtb_opt=True # Do XTB optimization.
                         )
print('Final Functionalized Structure:')
view_structures(newmol2)

### In this tutorial we learned how to edit existing molecules with architector including how to:

**(A)** Load exisiting molecules in and prepare to edit.

**(B)** Remove atoms from a molecule.

**(C)** Add functional groups to existing molecules.